In [2]:
%pip install scipy==1.8.0 numpy==1.21.5
from rosbag2_reader_py import Rosbag2Reader

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Open ROS 2 Bag

**The file `rosbag2_reader_py.py` shall be in the same folder of the notebook.**

In [3]:
path = "../rosbag2_2025_11_05-10_08_34"

reader = Rosbag2Reader(path)
topics = reader.all_topics
topics

[ERROR] [1763407642.314225744] [rosbag2_storage]: Could not open '../rosbag2_2025_11_05-10_08_34' with 'sqlite3'. Error: Failed to read from bag: File '../rosbag2_2025_11_05-10_08_34' does not exist!
[ERROR] [1763407642.314256688] [rosbag2_storage]: Could not load/open plugin with storage id 'sqlite3'


RuntimeError: No storage could be initialized from the inputs.

## Get total number of messages in the bag

In [ ]:
tot_msgs = 0
for _ in reader:
    tot_msgs += 1

print(f"Total messages: {tot_msgs}")

## Select messages of specific topics

In [ ]:
tot_msgs = 0
reader.set_filter(["/odom"])
for _ in reader:
    tot_msgs += 1

print("After the filter is applyed: ", reader.selected_topics)
print(f"Total messages: {tot_msgs}")

reader.reset_filter() # if you want to read all messages after you set a filter
print("After the filter is reset: ", reader.selected_topics)

## Access only data of a given type

In this example you can see  how to access an `Odometry` message checking for its type.

Please, notice the difference between **recording time** and time reported in the **stamp**. This is because the message was generated at a time that does not coincide with the time the message was received and recorded. This difference in a real robot may be really small, in the order of microseconds, but for a simulation, as in the reported case, the time could be extremely different. **You shall always use `header.stamp` whenever it is available.**

In [ ]:
from rclpy.time import Time
from nav_msgs.msg import Odometry
for topic_name, msg, t in reader:
    print(f"Received message of type {type(msg).__name__} on topic {topic_name} recorded at time {t}")
    if type(msg) is Odometry:
        time = Time.from_msg(msg.header.stamp).nanoseconds
        print(f"Position (x, y) at time {time}: ({msg.pose.pose.position.x:.2f}, {msg.pose.pose.position.y:.2f})")
    break

## Interpolate data to compute metrics

In order to compute the metrics for `/odom` and `/ekf` topics, you have to compare the poses reported in these topic with the poses reported in topic `/ground_truth` in the same time instants. 

Since the data are generated from different nodes at different frequencies, the time of the various topics will be different. So, we need to interpolate ground truth data on the time scale of the topic we want to evaluate.

First of all, let us save relevant data from messages in some NumPy arrays. As you can see from the output, the number of points from the two topics is different.

In [ ]:
import numpy as np
from scipy.interpolate import interp1d

time_gt = []
gt_data = []
time_odom = []
odom_data = []

reader.set_filter(["/ground_truth", "/ekf"])

for topic_name, msg, t in reader:
    if topic_name == "/ground_truth":
        time_gt.append(Time.from_msg(msg.header.stamp).nanoseconds)
        gt_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))
    elif topic_name == "/ekf":
        time_odom.append(Time.from_msg(msg.header.stamp).nanoseconds)
        odom_data.append((msg.pose.pose.position.x, msg.pose.pose.position.y))

time_gt = np.array(time_gt)
gt_data = np.array(gt_data)
time_odom = np.array(time_odom)
odom_data = np.array(odom_data)

print(f"Ground truth points: {len(gt_data)}")
print(f"Odometry points: {len(odom_data)}")

Now, let us create an interpolating function using SciPy `interp1d`.

In [ ]:
gt_interpol = interp1d(time_gt, gt_data, axis=0, fill_value="extrapolate", kind="nearest")
gt_data_interp = gt_interpol(time_odom)
print(f"Interpolated ground truth points: {len(gt_data_interp)}")

Compute Mean Absolute Error between odometry data and interpolated ground truth. You can find already implemented metrics functionson Portale della Didattica (Lecture_notebooks/Gaussian_filters.zip/utils.py)

In [ ]:
np.mean(np.linalg.norm(odom_data - gt_data_interp, axis=1), axis=0)